# Managing AI Engines (Driverless AI, H2O)

This notebook is intended to help you get started managing your AI Engines in the H2O AI Cloud using Python.

* **Product Documentation:** https://h2oai.github.io/ai-engine-manager/
* **Python Documentation:** https://h2oai.github.io/ai-engine-manager/docs/py/clients/

In [1]:
import os

import h2o
import h2o_authn
import h2o_discovery
import h2o_engine_manager

## Securely connect

We first connect to the H2O AI Cloud by creating a token provider object. We can then use this object to log into AI Engine Manager and other APIs.

In [2]:
discovery = h2o_discovery.discover()

token_provider = h2o_authn.TokenProvider(
    refresh_token=os.getenv("H2O_CLOUD_CLIENT_PLATFORM_TOKEN"),
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
)

engine_manager = h2o_engine_manager.login(token_provider=token_provider)

## Managing Driverless AI

In [4]:
dai_manager = engine_manager.dai_engine_client

### View available versions

In [ ]:
dai_manager.list_all_versions()

### Create an engine
See the `3 AutoML` tutorial for what to do once you've connected to the AutoML platform. 

In [7]:
dai_engine = dai_manager.create_engine(
    display_name="My test engine",
)

dai_engine.wait()

In [ ]:
dai_engine

In [ ]:
print(dai_engine.login_url)

In [ ]:
dai = dai_engine.connect()
dai.server.version

### List my engines

In [ ]:
for engine in dai_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

### Download engine logs

In [ ]:
dai_engine.download_logs()[:1000] # show only first 1000 characters

### Pause an engine

You can pause an engine that is currently running. Pausing an engine shuts it down to save resources, it is similar to powering off a server. You will not loose any data and you can resume the engine at any time.

In [12]:
dai_engine.pause()
dai_engine.wait()

### Resume the engine

You can resume a paused engine by running:

In [13]:
dai_engine.resume()
dai_engine.wait()

### Delete the engine

When you no longer need an engine, you can terminate it. Once deleted, there is no way to restart the engine or access any data.

In [14]:
dai_engine.delete()
dai_engine.wait()

In [ ]:
for engine in dai_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

## Managing H2O-3

In [15]:
h2o_manager = engine_manager.h2o_engine_client

### View available versions

In [ ]:
h2o_manager.list_all_versions()

### Create a H2O engine
See the H2O-3 Tutorial for what to do once you've connected to the distributed ML platform. 

In [17]:
h2o_engine = h2o_manager.create_engine(
    display_name="My test engine",
    version="latest"
)

h2o_engine.wait()

In [ ]:
print(h2o_engine.login_url)

In [ ]:
h2o.connect(config=h2o_engine.get_connection_config())

### List my H2O engines

In [ ]:
for engine in h2o_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)

### Download H2O logs

In [ ]:
h2o_engine.download_logs()[:1000] # show only first 1000 characters

### Stop an engine
H2O engines cannot be resumed after they are stopped. Only turn off your cluster once you are permanately done with it.

In [ ]:
h2o_engine.delete()
h2o_engine.wait()

In [ ]:
for engine in h2o_manager.list_all_engines():
    print(engine.name, "\t", engine.state.name, "\t", engine.display_name)